In [2]:
import csv
import pandas as pd 
import torch
import torch.nn as nn


In [3]:
df_train = pd.read_csv('2022_data/train.csv')      #2022
df_val = pd.read_csv('2022_data/val.csv')      #2022
# df_test = pd.read_csv('hindi_train.csv')  #2023

In [4]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("summarization", model="csebuetnlp/mT5_m2o_hindi_crossSum")

c:\Users\SAUMIL\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
c:\Users\SAUMIL\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\convert_slow_tokenizer.py:550: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [5]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("csebuetnlp/mT5_m2o_hindi_crossSum")
model = AutoModelForSeq2SeqLM.from_pretrained("csebuetnlp/mT5_m2o_hindi_crossSum")

In [6]:
train_article = df_train['Article'].tolist()[:10]
train_summary = df_train['Summary'].tolist()[:10]

val_article = df_val['Article'].tolist()
val_summary = df_val['Summary'].tolist()


print(len(train_article))
print(len(train_summary))
max_len = 0
for i in train_article:
    max_len = max(max_len, len(i.split()))
print(max_len)
print(len(val_article))

10
10
689
569


In [7]:
train_article[0]

'भुवनेश्वर : भारत ने शनिवार को ओडिशा के बालासोर तट पर ‘अग्नि प्राइम’ मिसाइल का सफल परीक्षण किया। सरकारी अधिकारियों ने इसकी जानकारी दी। अग्नि प्राइम मिसाइल अग्नि सीरीज की नए जनरेशन वाली एडवांस मिसाइल है। इसकी मारक क्षमता 1000 से 2000 किलोमीटर के बीच है। मिसाइल को रक्षा अनुसंधान और विकास संगठन (DRDO) द्वारा डिजाइन और विकसित किया गया है। अग्नि प्राइम अल्\u200dट्रा मॉर्डन टेक्निक से लैस होने की वजह से बहुत हल्के वजन की मिसाइल है।आपको बता दें कि इसी सप्ताह भारत ‘सुपरसोनिक मिसाइल असिस्टेड टॉरपीडो सिस्टम’ का भी सफलतापूर्वक प्रक्षेपण किया था। इस मिसाइल का प्रक्षेपण ओडिशा तट से दूर अब्दुल कलाम द्वीप से किया गया था।'

In [8]:

def prepare_dataset(dataset):
    encodings = tokenizer(dataset['article'], truncation=True, padding='max_length', max_length=512)
    decodings = tokenizer(dataset['summary'], truncation=True, padding='max_length', max_length=128)
    
    # Add required keys for the model
    encodings['labels'] = decodings['input_ids']
    encodings['decoder_input_ids'] = decodings['input_ids']
    encodings['decoder_attention_mask'] = decodings['attention_mask']
    
    return encodings

#make a dataset of the articles and summaries
train_dataset = [{'article': article, 'summary': summary} for article, summary in zip(train_article, train_summary)]
val_dataset = [{'article': article, 'summary': summary} for article, summary in zip(val_article, val_summary)]

# Example usage
prepared_dataset = [prepare_dataset(data) for data in train_dataset]
prepared_dataset_val = [prepare_dataset(data) for data in val_dataset]

In [9]:
type(prepared_dataset)

list

In [10]:
# from transformers import MT5ForConditionalGeneration, AutoTokenizer
# max_length = 3000
# #declare a tensor
# input_tensors = []
# # for i in range(0,7957):
inputs = tokenizer(train_article[i], text_target=train_summary[i], return_tensors="pt",truncation=True,padding="max_length",max_length=max_length)
# #     input_tensors.append(inputs)

# train_encodings = tokenizer(train_article,text_target=train_summary, padding=True, truncation=True, return_tensors="pt",max_length=max_length)
# val_encodings = tokenizer(val_article,text_target=val_summary, padding=True, truncation=True, return_tensors="pt",max_length=max_length)

# # val_tensors = []
# # for i in range(0,569):
# #     inputs = tokenizer(val_article[i], text_target=val_summary[i], return_tensors="pt",truncation=True,padding="max_length",max_length=max_length)
# #     val_tensors.append(inputs)

# # outputs = model(**inputs)
# # loss = outputs.loss

In [11]:
# import torch
# train_dataset = torch.utils.data.TensorDataset(train_encodings["input_ids"], train_encodings["attention_mask"])
# val_dataset = torch.utils.data.TensorDataset(val_encodings["input_ids"], val_encodings["attention_mask"])

In [12]:
# type(train_encodings)
# print(train_encodings.keys())

In [13]:
# from transformers import TrainingArguments

# training_args = TrainingArguments(output_dir="test_trainer")

In [14]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [15]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch",num_train_epochs=1,no_cuda=True)

c:\Users\SAUMIL\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\training_args.py:1405: FutureWarning: using `no_cuda` is deprecated and will be removed in version 5.0 of 🤗 Transformers. Use `use_cpu` instead
  warnings.warn(


In [16]:
# import torch

# # Clear CUDA cache

# import gc
# del variables
# gc.collect()

In [17]:
import torch
from torch.utils.data import TensorDataset

# train_dataset = TensorDataset(*input_tensors)
# val_dataset = TensorDataset(*val_tensors)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=prepared_dataset,
    eval_dataset=prepared_dataset_val,
    compute_metrics=compute_metrics,
)

In [19]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

trainer.train()

100%|██████████| 2/2 [01:51<00:00, 47.99s/it] 

RuntimeError: [enforce fail at alloc_cpu.cpp:114] data. DefaultCPUAllocator: not enough memory: you tried to allocate 13317963776 bytes.